# Setup enviorment

In [1]:
#!pip install keras-tuner

In [2]:
# Data reading in Dataframe format and data preprocessing
import pandas as pd
from pandas import read_csv
from pandas import DataFrame
from pandas import concat

# Data Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Linear algebra operations
import numpy as np

# Machine learning models and preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing

# Deep learning
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import Sequential, layers, callbacks
from tensorflow.keras.layers import Dense, LSTM, Dropout, GRU, Bidirectional
import keras_tuner

# Epiweek
from epiweeks import Week, Year

# Date
from datetime import date as convert_to_date

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
embeddings = 'Embeddings/embeddings_autoencoder_1024features.csv'
labels = 'Tabular_data/Label_CSV_All_Municipality.csv'
MUNICIPALITY = 'Ibagué'

# Read Data

In [5]:
def epiweek_from_date(image_date):
    date = image_date.split('-')
    
    # Get year as int
    year = ''.join(filter(str.isdigit, date[0]))
    year = int(year)
    
    # Get month as int
    month = ''.join(filter(str.isdigit, date[1]))
    month = int(month)
    
    # Get day as int
    day = ''.join(filter(str.isdigit, date[2]))
    day = int(day)
    
    # Get epiweek:
    date = convert_to_date(year, month, day)
    epiweek = str(Week.fromdate(date))
    epiweek = int(epiweek)
    
    return epiweek

### 1. Features

In [6]:
def read_features(path, Municipality = None):
    df = pd.read_csv(path)
    #df.Date = pd.to_datetime(df.Date)
    
    if Municipality:
        print('Obtaining dataframe for the city of Medellin only...')
        df = df[df['Municipality Code'] == Municipality]
        
    df.Date = df.Date.apply(epiweek_from_date)
    
    df = df.sort_values(by=['Date'])
    
    df = df.set_index('Date')
    
    if Municipality:
        df.drop(columns=['Municipality Code'], inplace=True)
        
    df.index.name = None
    return df

In [7]:
features_df = read_features(path=embeddings, Municipality='Medellín')
features_df

Obtaining dataframe for the city of Medellin only...


,0,1,2,3,4,5,6,7,8,9,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
201544,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,...,0.0,172.602520,0.0,0.00000,0.0,0.0,9.285753,68.562750,0.000000,0.0
201545,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,...,0.0,172.602520,0.0,0.00000,0.0,0.0,9.285753,68.562750,0.000000,0.0
201546,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,12.446627,0.000000,...,0.0,274.144930,0.0,0.00000,0.0,0.0,15.671944,2.534543,0.000000,0.0
201547,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,12.446627,0.000000,...,0.0,274.144930,0.0,0.00000,0.0,0.0,15.671944,2.534543,0.000000,0.0
201548,0.0,3.889202,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,...,0.0,0.000000,0.0,0.00000,0.0,0.0,4.791192,3.796963,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201848,0.0,0.000000,0.0,0.0,0.000000,0.0,21.014330,0.0,20.102703,0.469093,...,0.0,177.102700,0.0,4.91719,0.0,0.0,0.000000,0.000000,5.318927,0.0
201849,0.0,16.103472,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,...,0.0,31.717075,0.0,0.00000,0.0,0.0,0.000000,21.635908,0.000000,0.0
201850,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,...,0.0,80.470200,0.0,0.00000,0.0,0.0,0.000000,22.293388,0.000000,0.0
201851,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,...,0.0,0.000000,0.0,0.00000,0.0,0.0,0.000000,27.029966,0.000000,0.0


### 2. Labels

In [8]:
def get_epiweek(name):
    
    # Get week
    week = name.split('/')[1]
    week = week.replace('w','')
    week = int(week)
    
    # Year
    year = name.split('/')[0]
    year = int(year)
    
    epiweek = Week(year, week)
    
    epiweek = str(epiweek)
    epiweek = int(epiweek)

    return epiweek

In [9]:
def read_labels(path, Municipality = None):
    df = pd.read_csv(path)
    if df.shape[1] > 678:
        df = pd.concat([df[['Municipality code', 'Municipality']], df.iloc[:,-676:]], axis=1)
        cols = df.iloc[:, 2:].columns
        new_cols = df.iloc[:, 2:].columns.to_series().apply(get_epiweek)
        df = df.rename(columns=dict(zip(cols, new_cols))) 
        
    if 'Label_CSV_All_Municipality' in path:
        # Get Columns
        df = df[['epiweek', 'Municipality code', 'Municipality', 'final_cases_label']]
        
        # change epiweek format
        df.epiweek = df.epiweek.apply(get_epiweek)
        
        # Remove duplicates
        df = df[df.duplicated(['epiweek','Municipality code','Municipality']) == False]
        
        # Replace Increase, decrease, stable to numerical:
        """
        - Stable = 0
        - Increased = 1 
        - Decreased = 2
        """
        df.final_cases_label = df.final_cases_label.replace({'Stable': 0, 'Increased': 1, 'Decreased': 2})
        
        # Create table
        df = df.pivot(index=['Municipality code', 'Municipality'], columns='epiweek', values='final_cases_label')

        # Reset Index:
        df = df.reset_index()
    
    if Municipality:
        df = df[df['Municipality'] == Municipality]
        df.drop(columns=['Municipality code'], inplace=True)
        df.rename(columns={'Municipality': 'Municipality Code'}, inplace=True)
    
        df = df.set_index('Municipality Code')
        df = df.T

        df.columns.name = None
        df.index.name = None
        
        df.columns = ['Labels']
    
    return df

In [10]:
labels_df = read_labels(path=labels, Municipality='Medellín')
labels_df_orig = labels_df
labels_df = pd.get_dummies(labels_df['Labels'])
labels_df

,0,1,2
201601,1,0,0
201602,0,1,0
201603,0,0,1
201604,1,0,0
201605,1,0,0
...,...,...,...
201848,1,0,0
201849,0,0,1
201850,0,1,0
201851,1,0,0


# Data preparation

In [11]:
n_labels = labels_df.shape[1]

In [12]:
# Merge the two dataframes based on the date values
dengue_df = features_df.merge(labels_df, how='inner', left_index=True, right_index=True)
dengue_df

,0,1,2,3,4,5,6,7,8,9,...,1017,1018,1019,1020,1021,1022,1023,0,1,2
201601,0.0,0.000000,0.0,0.0,0.000000,0.0,8.161131,0.0,10.415243,0.000000,...,0.000000,0.0,0.0,7.435628,12.417972,0.000000,0.0,1,0,0
201602,0.0,0.000000,0.0,0.0,0.000000,0.0,8.161131,0.0,10.415243,0.000000,...,0.000000,0.0,0.0,7.435628,12.417972,0.000000,0.0,0,1,0
201603,0.0,0.000000,0.0,0.0,17.168774,0.0,43.288970,0.0,43.422768,15.177656,...,21.384554,0.0,0.0,1.990702,9.009418,26.557423,0.0,0,0,1
201604,0.0,0.000000,0.0,0.0,0.000000,0.0,1.034701,0.0,7.108294,0.000000,...,0.000000,0.0,0.0,2.045272,12.347998,0.000000,0.0,1,0,0
201605,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,...,0.000000,0.0,0.0,15.193535,6.474056,0.000000,0.0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201848,0.0,0.000000,0.0,0.0,0.000000,0.0,21.014330,0.0,20.102703,0.469093,...,4.917190,0.0,0.0,0.000000,0.000000,5.318927,0.0,1,0,0
201849,0.0,16.103472,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,...,0.000000,0.0,0.0,0.000000,21.635908,0.000000,0.0,0,0,1
201850,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,...,0.000000,0.0,0.0,0.000000,22.293388,0.000000,0.0,0,1,0
201851,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,...,0.000000,0.0,0.0,0.000000,27.029966,0.000000,0.0,1,0,0


### Train Test split

In [13]:
def train_test_split(df, train_percentage = 80):
    # We need a sequence so we can't split randomly
    # To divide into Train and test we have to calculate the train percentage of the dataset:
    size = df.shape[0]
    split = int(size*(train_percentage/100))
    
    """ Train """
    # We will train with 1st percentage % of data and test with the rest
    train_df = df.iloc[:split,:] ## percentage % train
    
    """ Test """
    test_df = df.iloc[split:,:] # 100 - percentage % test
    
    print(f'The train shape is: {train_df.shape}')
    print(f'The test shape is: {test_df.shape}')
    
    return train_df, test_df

In [14]:
train_df, test_df = train_test_split(dengue_df, train_percentage = 80)

The train shape is: (124, 1027)
The test shape is: (32, 1027)


### Normalize features

In [15]:
# Normalize train data and create the scaler
def normalize_train_features(df, feature_range=(-1, 1), n_labels=None):
    
    if n_labels:
        n_features = df.shape[1] - n_labels
    
    scalers = {}
    # For each column in the dataframe
    for i, column in enumerate(df.columns):
        if n_labels:
            if i >= n_features:
                break
        # Get values of the column
        values = df[column].values.reshape(-1,1)
        # Generate a new scaler
        scaler = MinMaxScaler(feature_range=feature_range)
        # Fit the scaler just for that column
        scaled_column = scaler.fit_transform(values)
        # Add the scaled column to the dataframe
        scaled_column = np.reshape(scaled_column, len(scaled_column))
        df[column] = scaled_column
        
        # Save the scaler of the column
        scalers['scaler_' + column] = scaler
        
    print(f' Min values are: ')
    print(df.min())
    print(f' Max values are: ')
    print(df.max())
        
    return df, scalers


""" If you want to use the same scaler used in train, you can use this function"""
def normalize_test_features(df, scalers=None, n_labels=None):
    
    if not scalers:
        raise TypeError("You should provide a list of scalers.")
    
    if n_labels:
        n_features = df.shape[1] - n_labels
    
    for i, column in enumerate(df.columns):
        if n_labels:
            if i >= n_features:
                break
        # Get values of the column
        values = df[column].values.reshape(-1,1)
        # Take the scaler of that column
        scaler = scalers['scaler_' + column]
        # Scale values
        scaled_column = scaler.transform(values)
        scaled_column = np.reshape(scaled_column,len(scaled_column))
        # Add the scaled values to the df
        df[column] = scaled_column
        
    print(f' Min values are: ')
    print(df.min())
    print(f' Max values are: ')
    print(df.max())
        
    return df 

In [16]:
feature_range = (-1, 1)

# Scale train:
train_df, scalers = normalize_train_features(train_df, feature_range=feature_range, n_labels=n_labels)

#print(f'The scalers are: {scalers}')

train_df.head()

 Min values are: 
0      -1.0
1      -1.0
2      -1.0
3      -1.0
4      -1.0
       ... 
1022   -1.0
1023   -1.0
0       0.0
1       0.0
2       0.0
Length: 1027, dtype: float64
 Max values are: 
0      -1.0
1       1.0
2      -1.0
3      -1.0
4       1.0
       ... 
1022    1.0
1023   -1.0
0       1.0
1       1.0
2       1.0
Length: 1027, dtype: float64


,0,1,2,3,4,5,6,7,8,9,...,1017,1018,1019,1020,1021,1022,1023,0,1,2
201601,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.0,-0.870525,-1.0,-0.814017,-1.000000,...,-1.000000,-1.0,-1.0,-0.849838,-0.843321,-1.000000,-1.0,1,0,0
201602,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.0,-0.870525,-1.0,-0.814017,-1.000000,...,-1.000000,-1.0,-1.0,-0.849838,-0.843321,-1.000000,-1.0,0,1,0
201603,-1.0,-1.0,-1.0,-1.0,-0.518855,-1.0,-0.313230,-1.0,-0.224607,-0.379313,...,-0.398361,-1.0,-1.0,-0.959798,-0.886327,-0.281606,-1.0,0,0,1
201604,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.0,-0.983585,-1.0,-0.873068,-1.000000,...,-1.000000,-1.0,-1.0,-0.958696,-0.844204,-1.000000,-1.0,1,0,0
201605,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.0,-1.000000,-1.0,-1.000000,-1.000000,...,-1.000000,-1.0,-1.0,-0.693167,-0.918316,-1.000000,-1.0,1,0,0


In [17]:
# Scale test:
test_df = normalize_test_features(test_df, scalers=scalers, n_labels=n_labels)
test_df.head()

 Min values are: 
0      -1.0
1      -1.0
2      -1.0
3      -1.0
4      -1.0
       ... 
1022   -1.0
1023   -1.0
0       0.0
1       0.0
2       0.0
Length: 1027, dtype: float64
 Max values are: 
0      -1.000000
1      -0.012869
2      -1.000000
3      -1.000000
4      -0.053137
          ...   
1022    0.305472
1023   -1.000000
0       1.000000
1       1.000000
2       1.000000
Length: 1027, dtype: float64


,0,1,2,3,4,5,6,7,8,9,...,1017,1018,1019,1020,1021,1022,1023,0,1,2
201821,-1.0,-1.0,-1.0,-1.0,-0.658391,-1.0,-0.300150,-1.0,-0.298219,-0.342267,...,-0.498781,-1.0,-1.0,-1.000000,-0.861126,-0.400018,-1.0,1,0,0
201822,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.0,-1.000000,-1.0,-1.000000,-1.000000,...,-1.000000,-1.0,-1.0,-1.000000,-0.971721,-1.000000,-1.0,1,0,0
201823,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.0,-1.000000,-1.0,-1.000000,-1.000000,...,-1.000000,-1.0,-1.0,-1.000000,-0.659987,-1.000000,-1.0,1,0,0
201824,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.0,-1.000000,-1.0,-1.000000,-1.000000,...,-1.000000,-1.0,-1.0,-1.000000,-1.000000,-1.000000,-1.0,1,0,0
201825,-1.0,-1.0,-1.0,-1.0,-0.268753,-1.0,-0.075702,-1.0,-0.065823,0.093075,...,-0.416461,-1.0,-1.0,-0.930679,-1.000000,-0.030467,-1.0,1,0,0


### Features and Labels Set

In [18]:
def features_labels_set(timeseries_data, original_df, n_labels):
    
    """ Features """
    # We define the number of features as (features and labels)
    n_features = original_df.shape[1]

    # The features to train the model will be all except the values of the actual week 
    # We can't use other variables in week t because whe need to resample a a 3D Array
    features_set = DataFrame(timeseries_data.values[:,:-n_labels])
    # Convert pandas data frame to np.array to reshape as 3D Array
    features_set = features_set.to_numpy()
    print(f'The shape of the features is {features_set.shape}')
    
    """ Labels """
    # We will use labels in last week 
    labels_set = DataFrame(timeseries_data.values[:,-n_labels:])
    # Convert pandas data frame to np.array
    labels_set = labels_set.to_numpy()
    print(f'The shape of the labels is {labels_set.shape}')
    
    return features_set, labels_set, n_features

In [19]:
# Train features and labels set
print('Train:')
train_X, train_y, n_features = features_labels_set(timeseries_data=train_df, original_df=dengue_df, n_labels=n_labels)

# Test features and labels set
print('Test:')
test_X, test_y, n_features = features_labels_set(timeseries_data=test_df, original_df=dengue_df, n_labels=n_labels)

Train:
The shape of the features is (124, 1024)
The shape of the labels is (124, 3)
Test:
The shape of the features is (32, 1024)
The shape of the labels is (32, 3)


# Modeling

# Define the Model

In [20]:
# Set Seed
#tf.random.set_seed(0)

def create_model():
    
    # design network
    model = Sequential()
    
    # Choose an optimal value between 32-512
    #hp_units = hp.Int('units', min_value=512, max_value=1024, step=64)
    model.add(Dense(units=1024))
    
    # Choose an optimal value between 32-512
    #hp_units_2 = hp.Int('units', min_value=32, max_value=512, step=32)
    model.add(Dense(units=512))
    
    # Out
    model.add(Dense(3, activation='softmax'))
    
    # Compile the model:
    
    # Choose an optimal value from 0.01, 0.001, or 0.0001
    #hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    opt = keras.optimizers.Adam()
    metrics = [
        tf.keras.metrics.AUC(name='auc', multi_label=False), #num_labels=3),
        tf.keras.metrics.CategoricalAccuracy(name='acc'),
        tf.keras.metrics.CategoricalCrossentropy(name='entropy')
    ]
    
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=metrics)

    return model

In [21]:
def tune_model():
    tuner = keras_tuner.RandomSearch(
        create_model,
        objective='val_loss',
        max_trials=15
    )
    
    tuner.search(
        train_X,
        train_y,
        epochs=1,
        batch_size=16,
        validation_data=(test_X, test_y)
    )

    return tuner

### Train the model

In [22]:
from tensorflow.keras.callbacks import EarlyStopping

# EarlyStopping:
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=20, 
        verbose=1, mode='auto', restore_best_weights=True)

In [23]:
### Imbalanced data
n_zeros = (labels_df_orig.to_numpy() == 0).sum()
n_ones = (labels_df_orig.to_numpy() == 1).sum()
n_twos = (labels_df_orig.to_numpy() == 2).sum()
n_total = n_zeros + n_ones + n_twos

weights = {0: n_total/n_zeros, 1: n_total/n_ones, 2: n_total/n_twos}
print(f'zeros: {n_zeros}, ones: {n_ones}, twos: {n_twos}, total: {n_total}')
weights

zeros: 113, ones: 20, twos: 23, total: 156


{0: 1.3805309734513274, 1: 7.8, 2: 6.782608695652174}

In [24]:
# fit network
def train_model(model, monitor, weights, plot=None, epochs=100):
    if monitor and weights:
        history = model.fit(train_X, train_y, epochs=epochs, batch_size=16, validation_data=(test_X, test_y), verbose=2, shuffle=False, callbacks=[monitor], class_weight=weights)
    elif monitor:
        history = model.fit(train_X, train_y, epochs=epochs, batch_size=16, validation_data=(test_X, test_y), verbose=2, shuffle=False, callbacks=[monitor])
    elif weights:
        history = model.fit(train_X, train_y, epochs=epochs, batch_size=16, validation_data=(test_X, test_y), verbose=2, shuffle=False, class_weight=weights)
    else:
        history = model.fit(train_X, train_y, epochs=epochs, batch_size=16, validation_data=(test_X, test_y), verbose=2, shuffle=False)
    
    if plot:
        # plot history
        plt.plot(history.history['loss'], label='train')
        plt.plot(history.history['val_loss'], label='validation')
        plt.legend()
        plt.show()
        

# Test the model

# AUC

In [25]:
# You can also evaluate or predict on a dataset.
def evaluate(model, verbose = None):
    if verbose:
        print('Evaluate: ')
    result = model.evaluate(test_X, test_y)
    stored_results = {}
    for i, metric in enumerate(model.metrics_names):
        stored_results[metric] = result[i]
        if verbose:
            print(f'{metric}: {result[i]}')
    return stored_results

# Calculate Mean and SD

In [26]:
def calculate_mean_std(weights):
    
    metrics = {
    "auc": [],
    "acc": [],
    "entropy": []
    }

    for i in range(5):
        model = create_model()
        train_model(model=model, monitor=monitor, weights=weights)
        stored_results = evaluate(model=model)
        
        for key in metrics.keys():
            metrics[key].append(stored_results[key])
            
    for key in metrics.keys():
        results = metrics[key]
        print(key, f": average={np.average(results):.3f}, std={np.std(results):.3f}")



In [27]:
calculate_mean_std(weights=None)

Epoch 1/100
8/8 - 1s - loss: 19.3899 - auc: 0.5508 - acc: 0.3710 - entropy: 19.3899 - val_loss: 7.4349 - val_auc: 0.9297 - val_acc: 0.9062 - val_entropy: 7.4349
Epoch 2/100
8/8 - 0s - loss: 21.9682 - auc: 0.4850 - acc: 0.3710 - entropy: 21.9682 - val_loss: 3.6793 - val_auc: 0.9297 - val_acc: 0.9062 - val_entropy: 3.6793
Epoch 3/100
8/8 - 0s - loss: 10.4733 - auc: 0.6881 - acc: 0.5161 - entropy: 10.4733 - val_loss: 0.6420 - val_auc: 0.9387 - val_acc: 0.8750 - val_entropy: 0.6420
Epoch 4/100
8/8 - 0s - loss: 2.3921 - auc: 0.7110 - acc: 0.5161 - entropy: 2.3921 - val_loss: 1.3573 - val_auc: 0.9297 - val_acc: 0.9062 - val_entropy: 1.3573
Epoch 5/100
8/8 - 0s - loss: 5.9310 - auc: 0.7567 - acc: 0.5403 - entropy: 5.9310 - val_loss: 2.2851 - val_auc: 0.9080 - val_acc: 0.9062 - val_entropy: 2.2851
Epoch 6/100
8/8 - 0s - loss: 5.1703 - auc: 0.7890 - acc: 0.6371 - entropy: 5.1703 - val_loss: 0.8890 - val_auc: 0.9253 - val_acc: 0.9062 - val_entropy: 0.8890
Epoch 7/100
8/8 - 0s - loss: 2.4207 - au

In [28]:
calculate_mean_std(weights=weights)

Epoch 1/100
8/8 - 1s - loss: 87.2224 - auc: 0.4824 - acc: 0.3145 - entropy: 28.0897 - val_loss: 23.1546 - val_auc: 0.2734 - val_acc: 0.0312 - val_entropy: 23.1546
Epoch 2/100
8/8 - 0s - loss: 100.9792 - auc: 0.3770 - acc: 0.1694 - entropy: 27.6703 - val_loss: 18.8518 - val_auc: 0.2969 - val_acc: 0.0625 - val_entropy: 18.8518
Epoch 3/100
8/8 - 0s - loss: 11.3218 - auc: 0.7844 - acc: 0.6613 - entropy: 2.8465 - val_loss: 1.0183 - val_auc: 0.9033 - val_acc: 0.8750 - val_entropy: 1.0183
Epoch 4/100
8/8 - 0s - loss: 19.0414 - auc: 0.2893 - acc: 0.1532 - entropy: 7.5693 - val_loss: 3.2131 - val_auc: 0.4094 - val_acc: 0.0625 - val_entropy: 3.2131
Epoch 5/100
8/8 - 0s - loss: 8.3975 - auc: 0.4397 - acc: 0.2419 - entropy: 3.2393 - val_loss: 2.6930 - val_auc: 0.4622 - val_acc: 0.0625 - val_entropy: 2.6930
Epoch 6/100
8/8 - 0s - loss: 7.1721 - auc: 0.4382 - acc: 0.1613 - entropy: 2.0218 - val_loss: 0.7654 - val_auc: 0.8845 - val_acc: 0.8125 - val_entropy: 0.7654
Epoch 7/100
8/8 - 0s - loss: 5.2491